# wind scenarios for buildout ACTIVSg grids

* conda: buildouts

* buildouts with new wind generators
* reading pre-made tables generated  part1 & part2 notebooks (save_dir)
* tamu200_buildout_part1.ipynb
* tamu200_buildout_part2.ipynb

#### ignore below buttet points:
* JHub:/projects/ecp/powerscenarios/notebooks/eagle_dev/tamu200_buildout_part1.ipynb
* JHub:/projects/ecp/powerscenarios/notebooks/eagle_dev/tamu200_buildout_part2.ipynb
* local mac: conda activate py3n
* /Users/isatkaus/projects/ecp/public-github/powerscenarios/notebooks/dev/buildouts/
* eagle: conda rtpv-eagle
* JHub: /home/isatkaus/projects/ecp/powerscenarios/notebooks/eagle_dev/
* similar to:
* projects/ecp/public-github/powerscenarios/notebooks/dev/buildouts/buildout_scenarios.ipynb


In [ ]:
!pwd

In [1]:
ll2 = ["blet", "test", "black",]

In [ ]:
import cufflinks as cl
import pandas as pd
import numpy as np
import time
import os
import matplotlib.pyplot as plt

# # set PYWTK_CACHE_DIR to locate WIND Toolkit data
# # will download from AWS as needed
# os.environ["PYWTK_CACHE_DIR"] = os.path.join(os.environ["HOME"], "pywtk-data")

from powerscenarios.parser import Parser
# from powerscenarios.grid import Grid
from powerscenarios.grid_copy2 import Grid

# show multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 100)

# plotting (optional)

cl.go_offline()

In [ ]:
%load_ext autoreload
%load_ext nb_black
%autoreload 2

# parse TAMU grid .aux files
* these are original grid files
* later we'll read modified grid files

In [ ]:
# choose TAMU grid
# grid files can be downloaded from:
# https://electricgrids.engr.tamu.edu/electric-grid-test-cases/

grid_name = "ACTIVSg200"  # TAMU 200 bus case
# grid_name = "ACTIVSg2000"  # TAMU 2000 bus case
# grid_name = "ACTIVSg10k"  # TAMU 10000 bus case

# path to .aux file (TAMU grids) obtained from e.g.
# https://electricgrids.engr.tamu.edu/electric-grid-test-cases/activsg200/
# local:
# data_dir = "../../../data/grid-data/"
# on Eagle/Kestrel:
data_dir = "/projects/hpcapps/isatkaus/grid-buildouts-data/grid-data/"
# aux_file_name = data_dir + grid_name + "/" + grid_name + ".aux"
aux_file_name = os.path.join(data_dir, grid_name, grid_name + ".aux")

# parse original .aux file and return dataframes for buses, generators, and wind generators
# here, we need .aux files because those are the only ones with Latitute/Longitude information
parser = Parser()
bus_df, gen_df, wind_gen_df = parser.parse_tamu_aux(aux_file_name)

# see what you got
print("bus_df:")
bus_df.head()
print("gen_df:")
gen_df.head()
print("wind_gen_df:")
wind_gen_df

print("\n original grid:")
grid = Grid(grid_name, bus_df, gen_df, wind_gen_df)
grid
print(grid.info())

# or parse RTS grid .csv files

In [ ]:
# # RTS-GMLC grid
# # https://github.com/GridMod/RTS-GMLC
# grid_name = "RTS"

# data_dir = "../data/grid-data"

# bus_csv_filename = os.path.join(data_dir, grid_name, "bus.csv")
# gen_csv_filename = os.path.join(data_dir, grid_name, "gen.csv")

# parser = Parser()

# # if solar2wind, will replace all solar to wind
# bus_df, gen_df, wind_gen_df = parser.parse_rts_csvs(
#     bus_csv_filename, gen_csv_filename, solar2wind=False
# )

# # see what you got
# print("bus_df:")
# bus_df.head()
# print("gen_df:")
# gen_df.head()
# print("wind_gen_df:")
# wind_gen_df.head()

## read pre-made tables and modified grid files
* chose the buildout configuration by setting "buildout_dir"
* buildout tables/grid files generated on Eagle with:
* tamu200_buildout_part1.ipynb
* tamu200_buildout_part2.ipynb

In [ ]:
###################################################################
# read instead of retrieve_wind_sites
###################################################################

# ### tables with Pmax from .m files
# table_dir = "/Users/isatkaus/projects/ecp/public-github/powerscenarios/data/tables-data/m-file-Pmax/"


# buildout dir: choose level 1,2,4,6,8, or 10
# buildout_level = 1
# ###
# table_dir = "/Users/isatkaus/projects/ecp/public-github/powerscenarios/data/tables-data/buildouts/orig_wind_locs/nameplate_x_{}/".format(
#     buildout_level
# )

########################################################
# CHOOSE BUILDOUT
# buildout_dir (contains tables and grid files )
# buildout_name = "ngens89-pen69"
# buildout_name = "option2_seed2-ngens11-pen86"
buildout_name = "option2_seed2-ngens10-pen89"
# buildout_name = "option2_seed2-ngens12-pen134"
# ### local
# buildout_dir = f"/Users/isatkaus/projects/ecp/public-github/powerscenarios/data/tables-data/buildouts/{buildout_name}"

# Eagle/Kestrel
# buildout_dir = f"/projects/exasgd/isatkaus/buildout/wind/wind_gen_tamu5/buildouts/work-dir/{buildout_name}"
buildout_dir = f"/projects/hpcapps/isatkaus/grid-buildouts-data/rev-runs/wind-gen-tamu200-v8/buildouts/work-dir/{buildout_name}"


# if using modified wind gens (new generators) also need
# new: bus_df, gen_df and wind_gen_df
filename = "bus_df.csv"
bus_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\nbus_df:")
bus_df

filename = "gen_df.csv"
gen_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\ngen_df:")
gen_df

filename = "wind_gen_df.csv"
wind_gen_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\nwind_gen_df:")
wind_gen_df


# to instantiate a grid we need: name, bus, generator, and wind generator dataframes from Parser
# really, we only wind generators, will change in the future
grid = Grid(grid_name, bus_df, gen_df, wind_gen_df)
grid
print(grid.info())

tables_dir = os.path.join(buildout_dir, "tables-data")

filename = "{}_wind_sites_df.h5".format(grid_name)
grid.wind_sites = pd.read_hdf(os.path.join(tables_dir, filename))
print("\n wind sites")
grid.wind_sites

# read instead of make_tables
filename = "{}_actuals_df.h5".format(grid_name)
grid.actuals = pd.read_hdf(os.path.join(tables_dir, filename))
print("\n actuals")
grid.actuals
filename = "{}_scenarios_df.h5".format(grid_name)
grid.scenarios = pd.read_hdf(os.path.join(tables_dir, filename))
print("\n scenarios")
grid.scenarios
##########################################################################

In [ ]:
# for actuals, make year you want: NOTE watch out for leap year!!!
# if actuals from tables are usually 2007, so if you change to 2020 you'll be missing
# Feb 29th will not exist
# if, on the other hand, actuals from tables are for 2008, then last day in December
# will be missing, since reV can only generate 365 days

##########################################################################
# for tamu200, load is in 2017, so we will remap actuals to 2017
# not a leap year, so everuthing should work as expected

grid.actuals.index = grid.actuals.index.map(lambda t: t.replace(year=2017))
# see what you got
print("\nactuals_df:")
grid.actuals
print("\nscenarios_df:")
grid.scenarios

# Select interesting time period for scenario generation
* e.g., find timestamps matching initial wind power on the network
* we want to match wind power Pg in .m file, else "ExaGO will not converge"
* find timestamps witihin delta, then random sample 
### can use dash-app for spatio-temporal viz

In [ ]:
############################################
# plotting total wind power of actuals
# to select sim_timestamps accordingly

t1 = pd.Timestamp("2017-07-01 00:15:00+0000", tz="UTC")
t2 = pd.Timestamp("2017-07-07 00:15:00+0000", tz="UTC")
grid.actuals.loc[t1:t2, "TotalPower"].iplot()

In [ ]:
# ### find actual date matching (within delta) the Pq power in the .m file

# print("\ntotal Pg (.m) wind power on the network:")
# total_Pg = wind_gen_m_df["Pg"].sum()
# total_Pg

# ## .m file has 12776 MW of wind
# ## to do sampling from mid bin 10%-90% we should pick total power below 12360
# total_Pg = 12100

# delta = 20
# sim_timestamp = (
#     grid.actuals.loc[
#         (grid.actuals["TotalPower"] > total_Pg - delta)
#         & (grid.actuals["TotalPower"] < total_Pg + delta)
#     ]
#     .sample()
#     .index[0]
# )

# sim_timestamp

# grid.actuals.loc[sim_timestamp:sim_timestamp]

# generate_wind_scenarios
* one timestamp
* outputs p_bin and cost_n
* full pipeline a few cell below

In [ ]:
t0 = time.time()
# time period for which to generate scenarios

# sim timestamp
sim_timestamp = pd.Timestamp("2017-07-01 11:25:00+0000", tz="UTC")


# other parameters
sampling_method = "monte carlo"
fidelity = None

# sampling_method = "importance"
# fidelity = "checkmark"

# fidelity = "exago_file"
n_scenarios = 10
n_periods = 6

# random_seed = np.random.randint(2 ** 31 - 1)
random_seed = 20
random_seed

scenarios_df, weights_df, p_bin, cost_n = grid.generate_wind_scenarios(
    sim_timestamp,
    power_quantiles=[0.0, 0.20, 0.80, 1.0],
    sampling_method=sampling_method,
    fidelity=fidelity,
    n_scenarios=n_scenarios,
    n_periods=n_periods,
    # random_seed=6,
    random_seed=random_seed,
    output_format=0,
)


# copy wanted actuals
actuals_df = (
    grid.actuals.loc[sim_timestamp:sim_timestamp].drop(
        "TotalPower", axis=1).copy()
)
# match index name to all_scenarios_df
actuals_df.index.name = "sim_timestamp"

print("\nactuals_df:")
actuals_df
print("\nscenarios_df:")
scenarios_df
print("\nweights_df:")
weights_df

print("\np_bin:")
p_bin

print("\ncost_n")
cost_n


t1 = time.time()

print("elapsed time = {}".format(t1 - t0))

In [ ]:
# start = pd.Timestamp("2008-01-03 00:07:30+00:00")
# end = pd.Timestamp("2008-01-04 00:09:00+00:00")
# dev_s = grid.scenarios.loc[cost_n.loc[start:end].index, "Deviation"]
# # dev_s
# cost_s = cost_n.loc[start:end]
# power_s = grid.scenarios.loc[cost_n.loc[start:end].index, "TotalPower"]
# # cost_s
# plot_df = pd.DataFrame(index=dev_s.index)
# plot_df["deviation"] = dev_s
# plot_df["cost"] = cost_s
# plot_df["power"] = power_s
# plot_df

## plot scenarios

In [ ]:
# total power
# choose sim_timestamp for which to plot all scenarios
# sim_timestamp = sim_timestamps[0]

# all needed period timestamps: t0,t1,...
timestamps = pd.date_range(
    start=sim_timestamp - pd.Timedelta("5min"), periods=n_periods + 1, freq="5min"
)
plot_df = pd.DataFrame(
    index=timestamps,
    columns=range(1, n_scenarios + 1),
)
for scenario_nr in range(1, n_scenarios + 1):
    s = scenarios_df.loc[
        (
            sim_timestamp,
            scenario_nr,
        )
    ].sum(axis=1)
    s.loc[timestamps[0]] = grid.actuals.loc[timestamps[0]].loc["TotalPower"]

    plot_df[scenario_nr] = s

# plot_df.iplot()
# plot_df.iplot(xTitle="Time", yTitle="MW", title="Total Wind Power", asImage=True,)
plot_df.iplot(xTitle="Time", yTitle="MW", title="Total Wind Power")

## generate scenarios for a period of time one buildout level
* come from buildout_scenarios_to_julia.ipynb (end of notebook)
* saving two cells below


In [ ]:
t0 = time.time()
# time period for which to generate scenarios

# # a few timestamps timestamp
# sim_timestamps = [
#     pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),
# ]
# sim_timestamps = [pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),pd.Timestamp("2020-07-01 00:20:00+0000", tz="UTC")]

# range
sim_timestamps = pd.date_range(
    start=pd.Timestamp("2017-06-01 00:00:00+0000", tz="UTC"),
    end=pd.Timestamp("2017-08-30 00:00:00+0000", tz="UTC"),
    freq="5min",
)


# other parameters
sampling_method = "monte carlo"
# sampling_method = "importance"
# fidelity = "checkmark"
n_scenarios = 1
n_periods = 1

########################################################

all_weights_df = pd.DataFrame(
    index=sim_timestamps, columns=range(1, n_scenarios + 1))

# create multiindex df for all generated scenarios
# three arrays for multiindex:
a1 = [x for x in sim_timestamps for k in range(n_scenarios * n_periods)]
a2 = [x for x in range(1, n_scenarios + 1) for k in range(n_periods)] * len(
    sim_timestamps
)
a3 = [
    t + pd.Timedelta("5min") * k
    for t in sim_timestamps
    for k in list(range(n_periods)) * n_scenarios
]

index = pd.MultiIndex.from_arrays(
    [a1, a2, a3], names=["sim_timestamp", "scenario_nr", "period_timestamp"]
)
all_scenarios_df = pd.DataFrame(
    index=index, columns=grid.wind_generators["GenUID"].values
)


for sim_timestamp in sim_timestamps:
    print("sim_timestamp = {}".format(sim_timestamp))
    random_seed = np.random.randint(2**31 - 1)
    # random_seed = 594081473
    print("random_seed = {}".format(random_seed))
    scenarios_df, weights_df, p_bin, cost_n = grid.generate_wind_scenarios(
        sim_timestamp,
        power_quantiles=[0.0, 0.20, 0.80, 1.0],
        sampling_method=sampling_method,
        fidelity=fidelity,
        n_scenarios=n_scenarios,
        n_periods=n_periods,
        # random_seed=6,
        random_seed=random_seed,
        output_format=0,
    )
    # all_scenarios_df=pd.concat([all_scenarios_df,scenarios_df])
    all_scenarios_df.loc[sim_timestamp] = scenarios_df

    # all_weights_df=pd.concat([all_weights_df,weights_df])
    all_weights_df.loc[sim_timestamp] = weights_df.loc[sim_timestamp]


# copy wanted actuals
all_actuals_df = grid.actuals.loc[sim_timestamps].drop(
    "TotalPower", axis=1).copy()
# match index name to all_scenarios_df
all_actuals_df.index.name = "sim_timestamp"

print("\nall_actuals_df:")
all_actuals_df
print("\nall_scenarios_df:")
all_scenarios_df
print("\nall_weights_df:")
all_weights_df

t1 = time.time()

print("elapsed time = {}".format(t1 - t0))

## sanity check

In [ ]:
print("\n actuals max:")
s1 = all_actuals_df.max()
s1 = s1.sort_index()
s1.index = s1.index.map(lambda x: x.replace("Wind", "WIND"))
s1
print("\n max cap of wind gens:")
s2 = grid.wind_generators.set_index("GenUID")["GenMWMax"]
s2.index = s2.index.map(lambda x: x.replace("Wind", "WIND"))
s2 = s2.sort_index()

print("\n any actuals > that GenMWMax?")
s1 > s2

print("\n max wind gen production is within 10% of wind gen capacities:")
abs((s1.sort_index() - s2.sort_index()).sum()) * 0.1 < s2.sum()

## save scenarios for one buildout level

In [ ]:
buildout_dir

In [ ]:
# for julia
# base_dir = "/Users/isatkaus/projects/ecp/julia/notebooks/buildouts/"

# save_dir = os.path.join(buildout_dir,"scenarios/month-of-july")
save_dir = os.path.join(buildout_dir, "scenarios/summer")

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

##############
# actuals
##############

# rename columns to match Maack's convention
all_actuals_df.index.rename("DateTime", inplace=True)
# all_actuals_df.drop("TotalPower",axis=1, inplace=True)

old_names = all_actuals_df.columns.values
new_names = [name.replace("Wind", "WIND") for name in old_names]
all_actuals_df.rename(columns=dict(zip(old_names, new_names)), inplace=True)

# drop tz, then julia's CSV can parse dates
all_actuals_df.index = all_actuals_df.index.map(
    lambda t: t.replace(tzinfo=None))

all_actuals_df

filename = "actuals_" + grid_name + ".csv"
all_actuals_df.to_csv(os.path.join(save_dir, filename))


##############
# scenarios
##############

old_names = all_scenarios_df.columns.values
new_names = [name.replace("Wind", "WIND") for name in old_names]
all_scenarios_df.rename(columns=dict(zip(old_names, new_names)), inplace=True)

# drop tz from scenarios as well, then julia's CSV can parse dates
all_scenarios_df.index = all_scenarios_df.index.map(
    lambda t: (t[0].replace(tzinfo=None), t[1], t[2].replace(tzinfo=None))
)
all_scenarios_df


if sampling_method == "monte carlo":
    method_str = "MC"
else:
    method_str = "IS"

filename = 'scenarios_{}_{}_{}per_{}scen.csv'.format(
    grid_name, method_str, n_periods, n_scenarios)
all_scenarios_df.to_csv(os.path.join(save_dir, filename))


######################################################
# also copy ACTIVSg200_m_gen.csv that was modified by the buildout
# and ACTIVSg200_m_bus.csv, ACTIVSg200_m_branch.csv that were not modified by buildout
# that way loading grid files in julia will be easy by pointing to the right dir
# ######################################################
# import shutil
# filename = "ACTIVSg200_m_gen.csv"
# ### table_dir = buildout_dir
# src = os.path.join(table_dir, filename)
# dst = os.path.join(save_dir, filename)
# shutil.copyfile(src, dst)

# orig_grid_data_dir = "/Users/isatkaus/projects/ecp/powerscenarios/data/grid-data/ACTIVSg200/"
# filename = "ACTIVSg200_m_bus.csv"
# src = os.path.join(orig_grid_data_dir, filename)
# dst = os.path.join(save_dir, filename)
# shutil.copyfile(src, dst)


# filename = "ACTIVSg200_m_branch.csv"
# src = os.path.join(orig_grid_data_dir, filename)
# dst = os.path.join(save_dir, filename)
# shutil.copyfile(src, dst)

In [ ]:
save_dir

# Scratch

In [ ]:
# total power
# choose sim_timestamp for which to plot all scenarios
# sim_timestamp = sim_timestamps[0]

# all needed period timestamps: t0,t1,...
timestamps = pd.date_range(
    start=sim_timestamp - pd.Timedelta("5min"), periods=n_periods + 1, freq="5min"
)
plot_df = pd.DataFrame(
    index=timestamps,
    columns=range(1, n_scenarios + 1),
)
for scenario_nr in range(1, n_scenarios + 1):
    s = scenarios_df.loc[
        (
            sim_timestamp,
            scenario_nr,
        )
    ].sum(axis=1)
    s.loc[timestamps[0]] = grid.actuals.loc[timestamps[0]].loc["TotalPower"]

    plot_df[scenario_nr] = s

# plot_df.iplot()
# plot_df.iplot(xTitle="Time", yTitle="MW", title="Total Wind Power", asImage=True,)
plot_df.iplot(xTitle="Time", yTitle="MW", title="Total Wind Power")

In [ ]:
# power at generators
# choose sim_timestamp and scenario nr for which to plot power at generators
# sim_timestamp = sim_timestamps[0]
scenario_nr = 5

# all needed period timestamps: t0,t1,...
timestamps = pd.date_range(
    start=sim_timestamp - pd.Timedelta("5min"), periods=n_periods + 1, freq="5min"
)

plot_df = scenarios_df.loc[(sim_timestamp, scenario_nr)]
plot_df.loc[timestamps[0]] = grid.actuals.loc[timestamps[0]]
plot_df.sort_index(inplace=True)

# plot_df.iplot()
# plot_df.iplot(xTitle="Time", yTitle="MW", title="scenario_nr={}".format(scenario_nr), asImage=True,)
plot_df.iplot(
    xTitle="Time",
    yTitle="MW",
    title="Power at generators when scenario_nr={}".format(scenario_nr),
)

## qcut problem

In [ ]:
# actuals: what percentage of time wind gen is at it's max?
actuals_df = grid.actuals
actuals_df

print("\n\nEXACTLY MAX POWER:")
# exactly maxes
for wind_farm in actuals_df.max().index:
    print(wind_farm)
    maxes_s = actuals_df.loc[actuals_df[wind_farm]
                             == actuals_df.max().loc[wind_farm]]

    print(
        "wind farm {} run at its max power {} fraction of the time ".format(
            wind_farm, round(len(maxes_s) / len(actuals_df), 4)
        )
    )

print("\n\nALMOST MAX POWER:")
# almost maxes

tol = 1

for wind_farm in actuals_df.max().index:
    print(wind_farm)
    almost_maxes_s = actuals_df[wind_farm].loc[
        abs(actuals_df[wind_farm] - actuals_df.max().loc[wind_farm]) < tol
    ]

    print(
        "wind farm {} run at its almost (within {}MW) max power {} fraction of the time ".format(
            wind_farm, tol, round(len(almost_maxes_s) / len(actuals_df), 4)
        )
    )


scenarios_df = grid.scenarios
scenarios_df
scenarios_df["TotalPower"].min()
total_max = scenarios_df["TotalPower"].max()
total_max

scenarios_df.loc[scenarios_df["TotalPower"] == total_max]

n_periods = 1
power_quantiles = [0.0, 0.15, 0.85, 1.0]
power_bins = pd.qcut(
    scenarios_df["TotalPower"].iloc[:-n_periods],
    q=power_quantiles,
)
power_bins

for power_bin in power_bins.cat.categories:
    power_bin
    scenarios_df.loc[power_bins.loc[power_bins == power_bin].index, "Deviation"].iplot(
        kind="hist", title="{}".format(power_bin)
    )

# save for Shri

In [ ]:
# actuals_df.drop(pd.Timestamp("2020-08-01 00:10:00+00:00"), axis=1, inplace=True)

In [ ]:
#!mkdir scenarios4shri

In [ ]:
# save_dir = "./scenarios4shri/checkmark/"
save_dir = "./scenarios4shri/checkmark/ACTIVSg10k/initial_wind_power_12100/"
# save_dir = "./scenarios4shri/monte_carlo/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
actuals_df.loc[sim_timestamp - pd.Timedelta("5min")] = grid.actuals.loc[
    sim_timestamp - pd.Timedelta("5min")
]
actuals_df.sort_index(inplace=True)


# save actuals_df
filename = "actuals_{}.csv".format(grid_name)
actuals_df.to_csv(os.path.join(save_dir, filename))

print("\nsaving actuals to {} ".format(os.path.join(save_dir, filename)))
actuals_df

In [ ]:
s_df = scenarios_df.loc[sim_timestamp]
# drop period_timestamp multi-index level
mindex = s_df.index
s_df.index = mindex.droplevel(1)

# add weights column
s_df["weight"] = weights_df.loc[sim_timestamp]


# save scenarios with weights
filename = "{}_scenarios_{}.csv".format(n_scenarios, grid_name)
s_df.to_csv(os.path.join(save_dir, filename))

print("\nsaving scenarios to {} ".format(os.path.join(save_dir, filename)))
s_df

## check if any scenario has a bus with wind power above Pmax of .m file

In [ ]:
wind_gen_df
max_power_s = wind_gen_df[["GenMWMax", "GenUID"]].set_index("GenUID")[
    "GenMWMax"]
max_power_s


print("\nany scenario has a bus with wind power above Pmax of .m file?")
(s_df.drop("weight", axis=1) > max_power_s).any().any()

s_df.drop("weight", axis=1) > max_power_s

In [ ]:
wind_gen_aux_df["GenUID"]
wind_gen_m_df["Pmax"]

In [ ]:
wind_gen_m_df.loc[77231]

In [ ]:
wind_gen_aux_df["GenUID"]

In [ ]:
# read scenarios
# grid_name = "ACTIVSg200"
n_scenarios = 10
filename = "{}_scenarios_{}.csv".format(n_scenarios, grid_name)
s_df = pd.read_csv(os.path.join(save_dir, filename), index_col=0)
s_df

In [ ]:
s_df.info()
s_df.loc[1, "65_Wind_1"]

In [ ]:
scenarios_df

In [ ]:
scenarios_df.loc[
    (
        pd.Timestamp("2020-08-01 00:15:00+00:00"),
        1,
        pd.Timestamp("2020-08-01 00:15:00+00:00"),
    )
]

In [ ]:
s_df.loc[1, "65_Wind_1"]
df = s_df.copy().round(decimals=grid.WTK_DATA_PRECISION)

# df.round(decimals=grid.WTK_DATA_PRECISION, inplace=True)
df.loc[1, "65_Wind_1"]
df

## generate_wind_scenarios

In [ ]:
t0 = time.time()
# time period for which to generate scenarios

# a few timestamps timestamp
sim_timestamps = [
    pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),
]
# sim_timestamps = [pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),pd.Timestamp("2020-07-01 00:20:00+0000", tz="UTC")]

# #range
# sim_timestamps = pd.date_range(
#    start=pd.Timestamp("2020-07-01 00:00:00+0000", tz="UTC"), end=pd.Timestamp("2020-07-07 00:00:00+0000", tz="UTC"), freq="5min"
# )


# other parameters
# sampling_method="monte carlo"
sampling_method = "importance"
fidelity = "checkmark"
# fidelity = "exago_file"
n_scenarios = 3
n_periods = 2

########################################################

all_weights_df = pd.DataFrame(
    index=sim_timestamps, columns=range(1, n_scenarios + 1))

# create multiindex df for all generated scenarios
# three arrays for multiindex:
a1 = [x for x in sim_timestamps for k in range(n_scenarios * n_periods)]
a2 = [x for x in range(1, n_scenarios + 1) for k in range(n_periods)] * len(
    sim_timestamps
)
a3 = [
    t + pd.Timedelta("5min") * k
    for t in sim_timestamps
    for k in list(range(n_periods)) * n_scenarios
]

index = pd.MultiIndex.from_arrays(
    [a1, a2, a3], names=["sim_timestamp", "scenario_nr", "period_timestamp"]
)
all_scenarios_df = pd.DataFrame(
    index=index, columns=grid.wind_generators["GenUID"].values
)


for sim_timestamp in sim_timestamps:
    # print("sim_timestamp = {}".format(sim_timestamp))
    random_seed = np.random.randint(2 ** 31 - 1)
    # random_seed = 594081473
    # print("random_seed = {}".format(random_seed))
    scenarios_df, weights_df, p_bin, cost_n = grid.generate_wind_scenarios(
        sim_timestamp,
        power_quantiles=[0.0, 0.1, 0.9, 1.0],
        sampling_method=sampling_method,
        fidelity=fidelity,
        n_scenarios=n_scenarios,
        n_periods=n_periods,
        # random_seed=6,
        random_seed=random_seed,
        output_format=0,
    )
    # all_scenarios_df=pd.concat([all_scenarios_df,scenarios_df])
    all_scenarios_df.loc[sim_timestamp] = scenarios_df

    # all_weights_df=pd.concat([all_weights_df,weights_df])
    all_weights_df.loc[sim_timestamp] = weights_df.loc[sim_timestamp]


# copy wanted actuals
all_actuals_df = grid.actuals.loc[sim_timestamps].drop(
    "TotalPower", axis=1).copy()
# match index name to all_scenarios_df
all_actuals_df.index.name = "sim_timestamp"

print("\nall_actuals_df:")
all_actuals_df
print("\nall_scenarios_df:")
all_scenarios_df
print("\nall_weights_df:")
all_weights_df

t1 = time.time()

print("elapsed time = {}".format(t1 - t0))

In [ ]:
!ls /Users/isatkaus/projects/ecp/public-github/powerscenarios/powerscenarios/costs/exago/opflowoptions


In [ ]:
!ls -lh

# plot 

In [ ]:
# total power
# choose sim_timestamp for which to plot all scenarios
# sim_timestamp = sim_timestamps[0]

# all needed period timestamps: t0,t1,...
timestamps = pd.date_range(
    start=sim_timestamp - pd.Timedelta("5min"), periods=n_periods + 1, freq="5min"
)
plot_df = pd.DataFrame(index=timestamps, columns=range(1, n_scenarios + 1),)
for scenario_nr in range(1, n_scenarios + 1):
    s = scenarios_df.loc[(sim_timestamp, scenario_nr,)].sum(axis=1)
    s.loc[timestamps[0]] = grid.actuals.loc[timestamps[0]].loc["TotalPower"]

    plot_df[scenario_nr] = s

# plot_df.iplot()
# plot_df.iplot(xTitle="Time", yTitle="MW", title="Total Wind Power", asImage=True,)
plot_df.iplot(xTitle="Time", yTitle="MW", title="Total Wind Power")

In [ ]:
# power at generators
# choose sim_timestamp and scenario nr for which to plot power at generators
# sim_timestamp = sim_timestamps[0]
scenario_nr = 5

# all needed period timestamps: t0,t1,...
timestamps = pd.date_range(
    start=sim_timestamp - pd.Timedelta("5min"), periods=n_periods + 1, freq="5min"
)

plot_df = scenarios_df.loc[(sim_timestamp, scenario_nr)]
plot_df.loc[timestamps[0]] = grid.actuals.loc[timestamps[0]]
plot_df.sort_index(inplace=True)

# plot_df.iplot()
# plot_df.iplot(xTitle="Time", yTitle="MW", title="scenario_nr={}".format(scenario_nr), asImage=True,)
plot_df.iplot(
    xTitle="Time",
    yTitle="MW",
    title="Power at generators when scenario_nr={}".format(scenario_nr),
)

# Importance sampling weights

$E_{f}\left[y\left(X\right)\right]=\int y\left(x\right)f(x)dx=\int y\left(x\right)\frac{f(x)}{g(x)}g(x)dx=E_{g}\left[y\left(X\right)\frac{f(X)}{g(X)}\right]$


$E\left[y\left(X\right)\right]\approx\frac{1}{N_{s}}{\displaystyle \sum_{i=1}^{N_{s}}y\left(X_{i}\right)} \;\;\; \textrm{with} \;\; X_i \; \textrm{from} \;\;f$


$E\left[y\left(X\right)\frac{f(X)}{g(X)}\right]\approx\frac{1}{N_{s}}{\displaystyle \sum_{i=1}^{N_{s}}y\left(X_{i}\right)\frac{f(X_i)}{g(X_i)}} \;\;\; \textrm{with} \;\; X_i \; \textrm{from} \;\;g$

$\frac{f(X_i)}{g(X_i)} = w_i \;\; \textrm{are importance sampling weights} $

# aggregate by bus (occasionally needed) 

In [ ]:
# all_actuals_df = all_actuals_df.groupby(lambda x: x.split("_")[0], axis=1).sum()
# all_scenarios_df = all_scenarios_df.groupby(lambda x: x.split("_")[0], axis=1).sum()
# all_actuals_df
# all_scenarios_df

# drop tz  (occasionally needed)

In [ ]:
# # drop tz from actuals
# all_actuals_df.index = all_actuals_df.index.map(lambda t: t.replace(tzinfo=None))

# # drop tz from scenarios
# all_scenarios_df.index=all_scenarios_df.index.map(
#     lambda t: (t[0].replace(tzinfo=None), t[1], t[2].replace(tzinfo=None))
# )

# all_actuals_df
# all_scenarios_df

# save as .csv

In [ ]:
# df = all_scenarios_df.copy()
# # if scenarios are single period, we can drop period_timestamp index level
# if n_periods == 1:
#     df.index=df.index.droplevel("period_timestamp")

# filename = './scenarios.csv'
# print("\nsaving all_scenarios_df to {}".format(filename))
# df
# df.to_csv(filename)

# # save weights
# filename = './weights.csv'
# print("\nsaving all_weights_df to {}".format(filename))
# all_weights_df
# all_weights_df.to_csv(filename)


# # take actuals corresponding to scenarios
# df=all_actuals_df
# df.index=df.index.rename("sim_timestamp")
# filename = './actuals.csv'
# print("\nsaving actuals to {}".format(filename))
# df
# df.to_csv(filename)

# saving as .aux 
will produce a separate .aux file for each actual and each senario period

In [ ]:
# # save_dir = "/Users/isatkaus/projects/ecp/powerscenarios/data/scenarios-data/tamara/"
# save_dir = "./aux/"


# ###################################### save scenarios
# for i in range(len(all_scenarios_df)):

#     s = all_scenarios_df.iloc[i]

#     # create filename from multiindex, option 1: <simulation timestamp>_<scenario number>_<period timestamp>
#     # filename = str(s.name[0]).replace(' ','-')+'_'+str(s.name[1])+'_'+str(s.name[2]).replace(' ','-')

#     # create filename from multiindex, option 2: <simulation timestamp>_<scenario number>_<period number>
#     filename = (
#         str(s.name[0]).replace(" ", "-")
#         + "_S"
#         + str(s.name[1])
#         + "_P"
#         + str(((s.name[2] - s.name[0]).seconds // 60) // 5 + 1)
#         + ".aux"
#     )

#     filename = save_dir + filename

#     delimiter = " "
#     delimiter = "\t"

#     with open(filename, "w") as f:
#         # .aux header
#         _ = f.write("DATA (Gen, [BusNum,GenID,GenMW,GenStatus])\n")
#         _ = f.write("{\n")
#         # each series entry is one line
#         for k in range(len(s)):
#             _ = f.write(
#                 delimiter
#                 + s.index[k].split("_")[0]
#                 + delimiter
#                 + '"'
#                 + s.index[k].split("_")[2]
#                 + '"'
#                 + delimiter
#                 + str(s[k])
#                 + delimiter
#                 + '"Closed"'
#                 + "\n"
#             )
#         # .aux EOF
#         _ = f.write("}\n")


# ######################################## save actuals

# for i in range(len(all_actuals_df)):

#     s = all_actuals_df.iloc[i]

#     # create filename from multiindex, option 2: <simulation timestamp>_<scenario number>_<period number>
#     filename = str(s.name).replace(" ", "-") + "_A" + ".aux"

#     filename = save_dir + filename

#     delimiter = " "
#     delimiter = "\t"

#     with open(filename, "w") as f:
#         # .aux header
#         _ = f.write("DATA (Gen, [BusNum,GenID,GenMW,GenStatus])\n")
#         _ = f.write("{\n")
#         # each series entry is one line
#         for k in range(len(s)):
#             _ = f.write(
#                 delimiter
#                 + s.index[k].split("_")[0]
#                 + delimiter
#                 + '"'
#                 + s.index[k].split("_")[2]
#                 + '"'
#                 + delimiter
#                 + str(s[k])
#                 + delimiter
#                 + '"Closed"'
#                 + "\n"
#             )
#         # .aux EOF
#         _ = f.write("}\n")

# Scratch